In [ ]:
from langchain.document_loaders import PDFMinerPDFasHTMLLoader
from bs4 import BeautifulSoup

In [ ]:
loader = PDFMinerPDFasHTMLLoader("Input/input.pdf")
data = loader.load()[0]

print(data)

In [15]:
soup = BeautifulSoup(data.page_content,'html.parser')
content = soup.find_all('div')

print(content)

[<div style="position:absolute; top:50px;"><a name="1">Page 1</a></div>, <div style="position:absolute; border: textbox 1px solid; writing-mode:lr-tb; left:135px; top:75px; width:66px; height:15px;"><span style="font-family: Arial-BoldMT; font-size:6px">Open Access Review
<br/>Article
<br/></span></div>, <div style="position:absolute; border: textbox 1px solid; writing-mode:lr-tb; left:302px; top:82px; width:83px; height:9px;"><span style="font-family: NimbusSans-Regular; font-size:9px"> </span><span style="font-family: Arial-BoldMT; font-size:6px">DOI:</span><span style="font-family: ArialMT; font-size:6px"> 10.7759/cureus.40274
<br/></span></div>, <div style="position:absolute; border: textbox 1px solid; writing-mode:lr-tb; left:83px; top:152px; width:60px; height:4px;"><span style="font-family: Arial-BoldMT; font-size:4px">Review began</span><span style="font-family: ArialMT; font-size:4px"> 05/30/2023 
<br/></span></div>, <div style="position:absolute; border: textbox 1px solid; wr

In [26]:
import re
cur_fs = None
cur_text = ''
snippets = []   # first collect all snippets that have the same font size
for c in content:
    sp = c.find('span')
    if not sp:
        continue
    st = sp.get('style')
    if not st:
        continue
    fs = re.findall('font-size:(\d+)px',st)
    if not fs:
        continue
    fs = int(fs[0])
    if not cur_fs:
        cur_fs = fs
    if fs == cur_fs:
        cur_text += c.text
    else:
        snippets.append((cur_text,cur_fs))
        cur_fs = fs
        cur_text = c.text
snippets.append((cur_text,cur_fs))

print(snippets)
# Note: The above logic is very straightforward. One can also add more strategies such as removing duplicate snippets (as
# headers/footers in a PDF appear on multiple pages so if we find duplicates it's safe to assume that it is redundant info)

c_text:Page 1
page 1
False
c_text:Open Access Review
Article

open access review
article

False
c_text: DOI: 10.7759/cureus.40274

 doi: 10.7759/cureus.40274

False
c_text:Review began 05/30/2023 

review began 05/30/2023 

False
c_text:Review ended 06/08/2023 

review ended 06/08/2023 

False
c_text:Published 06/11/2023

published 06/11/2023

False
c_text:© Copyright  2023

© copyright  2023

False
c_text:Saxena et al. This is an open access article

saxena et al. this is an open access article

False
c_text:distributed under the terms of the Creative

distributed under the terms of the creative

False
c_text:Commons Attribution License CC-BY 4.0.,

commons attribution license cc-by 4.0.,

False
c_text:which permits unrestricted use, distribution,

which permits unrestricted use, distribution,

False
c_text:and reproduction in any medium, provided

and reproduction in any medium, provided

False
c_text:the original author and source are credited.

the original author and source are cr

In [30]:
from langchain.docstore.document import Document
cur_idx = -1
semantic_snippets = []
# Assumption: headings have higher font size than their respective content
for s in snippets:
    if s[0].lower == 'references': break
    # if current snippet's font size > previous section's heading => it is a new heading
    if not semantic_snippets or s[1] > semantic_snippets[cur_idx].metadata['heading_font']:
        metadata={'heading':s[0], 'content_font': 0, 'heading_font': s[1]}
        metadata.update(data.metadata)
        semantic_snippets.append(Document(page_content='',metadata=metadata))
        cur_idx += 1
        continue

    # if current snippet's font size <= previous section's content => content belongs to the same section (one can also create
    # a tree like structure for sub sections if needed but that may require some more thinking and may be data specific)
    if not semantic_snippets[cur_idx].metadata['content_font'] or s[1] <= semantic_snippets[cur_idx].metadata['content_font']:
        semantic_snippets[cur_idx].page_content += s[0]
        semantic_snippets[cur_idx].metadata['content_font'] = max(s[1], semantic_snippets[cur_idx].metadata['content_font'])
        continue

    # if current snippet's font size > previous section's content but less than previous section's heading than also make a new
    # section (e.g. title of a PDF will have the highest font size but we don't want it to subsume all sections)
    metadata={'heading':s[0], 'content_font': 0, 'heading_font': s[1]}
    metadata.update(data.metadata)
    semantic_snippets.append(Document(page_content='',metadata=metadata))
    cur_idx += 1

In [29]:
print(len(semantic_snippets))
print(semantic_snippets[12])

13
page_content="1.  Karan A, Negandhi H, Hussain S, et al.:  Size, composition and distribution of health workforce in India:\n2. \nwhy, and where to invest?. Hum Resour Health. 2021, 19:39.  10.1186/s12960-021-00575-2\nIndian government doubles health budget after Covid exposes long-term underfunding . (2021). Accessed:\nOctober 12, 2021: http://<https://www.telegraph.co.uk/global-health/science-and-disease/indian-\ngovernment-doubles-health-budget-covid-e....\n3.  Dalglish SL, Khalid H, McMahon SA:  Document analysis in health policy research: the READ approach .\nHealth Policy Plan. 2021, 35:1424-31. 10.1093/heapol/czaa064\n4.  Physicians (per 1,000 people) - India . Accessed: October 12, 2021:\nhttps://data.worldbank.org/indicator/SH.MED.PHYS.ZS.\n5.  Health workforce requirements for universal health coverage and the sustainable development goals . (2016).\nAccessed: October 6, 2018: https://www.who.int/publications/i/item/9789241511407.\n6.  Kruk ME, Gage AD, Arsenault C, et al.